## 사전지식 

### 1. AutoML
- Machine Learning 으로 설계하는 Machine Learning을 의미하며 Automated Feature Learning / Architecture Search / Hyperparameter Optimization 3가지 방향으로 주로 연구되고 있음


- (1) Automated Feature Learning
    - 유의미한 Feature를 추출해 학습 모델의 입력으로 사용하는 방법
    
- (2). Architecture Search
    - Network에 대한 최적의 Architecture(구조)를 설계하는 방법. 주로 강화학습이나 유전 알고리즘 등을 이용한 연구들이 많이 이용되고 있음


- (3). Hyperparameter Optimization
     - 학습을 시키기 위한 Hyperparameter를 추정하는 것.
     - 기존에는 learning rate, mini-batch size 등 학습에 영향을 주는 hyperparameter들을 사람이 바꿔가며 모델을 학습시켜 사용


- (4). 유사 딥러닝에서의 AutoML연구 진행 방향
    - Optimizer : Neural Optimizer Search with Reinforcement Learning
    - Activation Function : Searching for Activation Functions (swish)
    - Augmentation : AutoAugment-Learning Augmentation Policies from Data
    - Mobile Device : MnasNet: Platform-Aware Neural Architecture Search for Mobile
    
### 2. NAS
- 강화학습 기반으로 최적의 architecture를 찾는 연구 : 2017_Neural Architecture Search with reinforcement learning
- NAS 구성 : (1) + (2) 
- (1) Network의 Architecture를 결정하는 요소들(filter size, stride 등)의 값을 예측하는 RNN Controller
- (2) RNN Controller가 출력한 값들로 구성된 모델을 학습시켜 validation accuracy를 reward로 얻은 강화학습 모델
- Cifar-10에 대한 최적 모델을 찾기까지 800대 GPU로 한달이 걸림

<hr>

## NASNet 본문
- Paper:(2018 CVPR)Learning Transferable Architectures for Scalable Image Recognition
- NASNet은 선행연구 NAS에서 발전된 연구이며, NAS와 다르게 CNN 구조 탐색의 범위를 한정 지어 실험
- 학습하는데 500 GPU, 4days가 소요 됬으며, Cifar-10 NAS 모델에서 잘되던 구조를 참고하여 탐색 범위 설정
- Search space의 변화
    - NAS처럼 전체 탐색 공간을 사용한 것이 아닌, 좋은 결과를 줄 몇개에 대해 일부 범위를 한정 지어놓고 실험 진행
- **Convolution cell이라는 단위를 사용하며, 전체 Network대신 이 Cell들을 탐색한 뒤, 이 Cell들을 조합하여 전체 Network를 설계**함
- 이전 SOTA인 사람이 디자인한 SENet에 비해 사용된 Parameter도 적고, 거의 비슷한 accuracy 달성

<hr>

## RNN Controller
- RNN controller는 one layer LSTM 구조
- 각 layer마다 100개의 hidden unit을 가짐.
- RNN controller를 학습시키기 위한 강화학습 알고리즘으로는 2017년 OpenAI에서 발표한 Proximal Policy Optimization(PPO)를 사용 (NAS는 Reinforce rule을 사용)

## Block 생성 과정
![rnn_controller](img/rnn_controller.png)

- Block : 2개의 연산을 수행하여 하나의 Feature map을 출력하는 역할
- 위의 RNN Controller 5개의 값이 하나의 block을 결정
- Hidden state input 
    - 해당 block이 포함되어 있는 convolution cell의 input의 선택 제약 조건
        - 해당 block이 포함되어 있는 convolution cell의 input인 hi
        - 이전 convolution cell의 input인 hi-1
        - 해당 block의 다른 hidden state의 output 중 하나를 선택
- Operation
    - 13가지가 있으며, 성능이 좋다고 알려져 있는 Operation 위주
        - Identity, Pooling, Conv, conv의 변형 등
- Combine Operation
    - 2개의 Operation으로 나온 값을 그대로 element-wise할지 아니면 channel-wise로 concatenate하여 사용할지 선택

<hr>

## Convolution Cell 
- **Normal Cell + Reduction Cell** 로 구성
- Normal, Reduction Cell 모두 Block들로부터 구성되어 생성되며, 유일한 차이는 Block내 연산들의 stride 차이임

- Normal Cell
    - 입력과 출력의 Feature map의 가로, 세로 크기가 같은 Cell
- Reduction Cell
    - 출력 Feature map의 가로, 세로 크기가 입력 Feature map의 가로, 세로 크기의 절반이 되는 Cell
    - stride값을 1또는 2를 사용
    
## Convolution Cell을 통한 전체 Architecture 구성 과정

![nasnet_convolution_cell](img/nasnet_convolution_cell.png)

- 1. RNN controller로부터 Normal Cell을 위한 block B개, Reduction Cell을 위한 block B개 생성
- 2. 생성된 2B개의 block을 이용하여, Normal Cell과 Reduction Cell 두 가지의 Cell을 구성(위 그림의 좌측 하단)
- 3. 두 가지의 Normal Cell, Reduction Cell을 정해진 순서에 맞게 배치하여 network를 구성
- 4. 구성된 network를 토대로 training data로 학습
- 5. 학습 후 validation accuracy(reward)를 측정 결과에 따라 강화학습 수행
- 6. RNN controller를 update후, 다시 1번으로 돌아가서 진행.

<hr>

## NAS Architecture vs NASNet Architecture

### NAS Architecture
![nas_cifar10_architecture](img/nas_cifar10_architecture.png)

- 3x3 conv는 3번밖에 쓰이지 않고, 5x5, 7x7, 5x7, 7x7등의 다양한 모양의 conv filter사용
- 또한 skip connection이 굉장히 많아 사람이 design하기에는 다소 무리가 있음


### NASNet_Architecture
- 본 논문에서는 성능이 좋은 3가지 Version 제안 (NASNet-A,B,C) (A가 가장 성능이 우수)

![nasnet_a_architecture](img/nasnet_a_architecture.png)

- 특징
    - Operation : depthwise-separable convolution (sep) 연산만 사용됨 
    - Combine Operation : 전부 add operation
    - Pooling : 거의 3x3 pooling이 선택됨
    - NASNet-B,C, 에서도 비슷한 경향을 보임 


## NASNet Result 실험 결과
- NASNet의 주요 강점 중 하나는 Transferability
- 비교적 이미지 사이즈와 개수가 적은 CIFAR-10에서 찾은 Convolution Cell을 이용하여 ImageNet 데이터셋에서 학습을 수행 -> 거의 차이가 없음에도 좋은 성능을 보임

![nasnet_cifar10_imagenet_architecture](img/nasnet_cifar10_imagenet_architecture.png)



- ImageNet실험 결과

![nasnet_imagenet_result](img/nasnet_imagenet_result.png)

- mobile device 타겟 실험결과

![nasnet_mobile_result](img/nasnet_mobile_result.png)

<hr>

## 참고문헌
- https://hoya012.github.io/blog/Learning-Transferable-Architectures-for-Scalable-Image-Recognition-Review/ // hoya님 깃헙 블로그